In [1]:
# This piece of code generates labels for the selected subset of clean images.
# Labels are extracted from the train.yaml with the help of image name (Bosch dataset follows global name)
# Import necessary libraries

import yaml
import os
from PIL import Image

def map_label(label):
    if label.startswith('Red'):
        return 0
    elif label.startswith('Green'):
        return 1
  #  elif label.startswith('Yellow'):
  #      return 2  # Optional
    else:
        return None  # Skip 'Off' and others

# Paths
image_dir = '/content/drive/MyDrive/TrafficSignal/images'
yaml_path = '/content/drive/MyDrive/TrafficSignal/train.yaml'

# Load YAML
with open(yaml_path, 'r') as f:
    bosch_data = yaml.safe_load(f)

# Track stats
num_found = 0
num_missing = 0

for image_file in os.listdir(image_dir):
    if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    full_image_path = os.path.join(image_dir, image_file)

    # Get image dimensions
    try:
        with Image.open(full_image_path) as img:
            img_w, img_h = img.size
    except:
        print(f" Could not open image: {image_file}")
        continue

    # Find matching entry in YAML
    found = False
    for entry in bosch_data:
       # if entry['path'].endswith(image_file):
        if os.path.basename(entry['path']) == image_file:

            label_path = full_image_path.replace('.png', '.txt').replace('.jpg', '.txt').replace('.jpeg', '.txt')

            with open(label_path, 'w') as out:
                for box in entry['boxes']:
                    cls = map_label(box['label'])
                    if cls is None:
                        continue

                    x_min = box['x_min']
                    y_min = box['y_min']
                    x_max = box['x_max']
                    y_max = box['y_max']

                    # Convert to YOLO format
                    x_center = ((x_min + x_max) / 2) / img_w
                    y_center = ((y_min + y_max) / 2) / img_h
                    box_w = (x_max - x_min) / img_w
                    box_h = (y_max - y_min) / img_h

                    out.write(f"{cls} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")

            num_found += 1
            found = True
            break

    if not found:
        num_missing += 1

print(f" Generated YOLO labels for {num_found} images.")
print(f" Skipped {num_missing} images (no annotations).")


 Generated YOLO labels for 663 images.
 Skipped 0 images (no annotations).


In [2]:
# This piece of code splits the dataset (subset) into train, val, and test
import os
import random
import shutil

# Paths
base_dir = '/content/drive/MyDrive/TrafficSignal'
image_dir = os.path.join(base_dir, 'images')
output_dirs = ['train', 'val', 'test']

# Create folders
for split in output_dirs:
    os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
image_files.sort()  # Optional: for reproducibility
random.seed(42)
random.shuffle(image_files)

# Split 70% train, 20% val, 10% test
n = len(image_files)
train_split = int(0.7 * n)
val_split = int(0.9 * n) # 0.9-0.7 = 0.2

splits = {
    'train': image_files[:train_split],
    'val': image_files[train_split:val_split],
    'test': image_files[val_split:]
}

# Move files
for split, files in splits.items():
    for file in files:
        image_src = os.path.join(image_dir, file)
        label_src = os.path.join(image_dir, file.rsplit('.', 1)[0] + '.txt')

        image_dst = os.path.join(base_dir, split, 'images', file)
        label_dst = os.path.join(base_dir, split, 'labels', os.path.basename(label_src))

        shutil.copy(image_src, image_dst)
        shutil.copy(label_src, label_dst)

print(" Split complete!")


 Split complete!


In [ ]:
# Install ultralytics

!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
# Train model by setting epoch, image size, batch size
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')  # or 'yolov8s.yaml' for slightly bigger
model.train(data='/content/drive/MyDrive/TrafficSignal/data.yaml', epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/TrafficSignal/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, kera

100%|██████████| 755k/755k [00:00<00:00, 22.6MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n summary: 129 layers, 3,011,238 parameters, 3,011,222 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 89.4MB/s]


...

                   all        132        256      0.794      0.614      0.647       0.34



50 epochs completed in 0.248 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.29it/s]


                   all        132        256      0.801       0.61      0.657      0.344
                   red         49        134      0.771      0.522      0.583      0.304
                 green         43        122      0.832      0.697       0.73      0.384
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd8be96c210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
!cp runs/detect/train/weights/best.pt /content/drive/MyDrive/TrafficSignal/best.pt


In [ ]:
# Save the resullts on test data for analysing (vary conf score)
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/TrafficSignal/best.pt")
results = model.predict(source="/content/drive/MyDrive/TrafficSignal/test/images", save=True, conf=0.25)



image 1/67 /content/drive/MyDrive/TrafficSignal/test/images/102520.png: 384x640 (no detections), 35.3ms
image 2/67 /content/drive/MyDrive/TrafficSignal/test/images/105038.png: 384x640 (no detections), 7.1ms
image 3/67 /content/drive/MyDrive/TrafficSignal/test/images/105176.png: 384x640 (no detections), 7.1ms
image 4/67 /content/drive/MyDrive/TrafficSignal/test/images/10786.png: 384x640 2 greens, 15.4ms
image 5/67 /content/drive/MyDrive/TrafficSignal/test/images/107954.png: 384x640 (no detections), 13.6ms
image 6/67 /content/drive/MyDrive/TrafficSignal/test/images/114386.png: 384x640 (no detections), 18.6ms
image 7/67 /content/drive/MyDrive/TrafficSignal/test/images/114524.png: 384x640 (no detections), 7.2ms
image 8/67 /content/drive/MyDrive/TrafficSignal/test/images/118832.png: 384x640 (no detections), 7.3ms
image 9/67 /content/drive/MyDrive/TrafficSignal/test/images/121174.png: 384x640 3 greens, 7.1ms
image 10/67 /content/drive/MyDrive/TrafficSignal/test/images/121382.png: 384x640 (n

In [ ]:
# Zio the result for download

!zip -r results.zip runs/detect/predict/

  adding: runs/detect/predict/ (stored 0%)
  adding: runs/detect/predict/99190.jpg (deflated 0%)
  adding: runs/detect/predict/215090.jpg (deflated 0%)
  adding: runs/detect/predict/217304.jpg (deflated 0%)
  adding: runs/detect/predict/21452.jpg (deflated 1%)
  adding: runs/detect/predict/140376.jpg (deflated 1%)
  adding: runs/detect/predict/140724.jpg (deflated 1%)
  adding: runs/detect/predict/37526.jpg (deflated 0%)
  adding: runs/detect/predict/616892.jpg (deflated 0%)
  adding: runs/detect/predict/68492.jpg (deflated 1%)
  adding: runs/detect/predict/114524.jpg (deflated 1%)
  adding: runs/detect/predict/70090.jpg (deflated 0%)
  adding: runs/detect/predict/65218.jpg (deflated 1%)
  adding: runs/detect/predict/87334.jpg (deflated 1%)
  adding: runs/detect/predict/105176.jpg (deflated 1%)
  adding: runs/detect/predict/212730.jpg (deflated 0%)
  adding: runs/detect/predict/102520.jpg (deflated 1%)
  adding: runs/detect/predict/616822.jpg (deflated 1%)
  adding: runs/detect/predict

In [ ]:
# Download the zipped results

from google.colab import files
files.download("results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the best model to export it in ONNX format

from ultralytics import YOLO
model = YOLO("runs/detect/train/weights/best.pt")


In [ ]:
# Export the model in ONNX format
# ONNX
model.export(format="onnx") # Interoperability, cloud services

# TensorRT (requires GPU and dependencies)
model.export(format="engine")  # .engine file High-speed inference on NVIDIA GPUs

# CoreML (for Apple devices)
model.export(format="coreml") # 	iOS devices

# Other supported: "torchscript", "openvino" for Intel devices, edge AI, "tfjs" to run in browser, "tflite" for Mobile Android apps, "pb" (TensorFlow)


Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8n summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.46', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 282.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 368.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 241.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 250.8 MB/s eta 0:00:00

requirements: Aut


CoreML: starting export with coremltools 8.3.0...


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 111.74 passes/s]


CoreML: export success ✅ 22.2s, saved as 'runs/detect/train/weights/best.mlpackage' (5.9 MB)

Export complete (22.7s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.mlpackage imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.mlpackage imgsz=640 data=/content/drive/MyDrive/TrafficSignal/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.mlpackage'

In [ ]:
# Download ONNX model

from google.colab import files
files.download("runs/detect/train/weights/best.onnx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>